## 1. Importacion De Librerias

Se importan las librerías necesarias para el entrenamiento de modelos: numpy y pandas para manipulación de datos, scikit-learn para los algoritmos de clasificación (KNN, SVM, Naive Bayes, Random Forest), joblib para serializar modelos entrenados, y os para manejo de rutas de archivos.

In [ ]:
# Librerías para manipulación de datos
import numpy as np
import pandas as pd

# Modelos de clasificación
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Manejo de rutas y archivos
import os

# Serialización de modelos entrenados
import joblib

## 2. Preparacion De Datos, Directorios y Modulos

Se definen las constantes del proyecto y la función `get_model()` que actúa como **factory pattern** para crear instancias frescas de modelos. Esto evita la reutilización de instancias entre folds, previniendo contaminación de estado. Se configuran 9 tipos de datasets, 4 modelos de clasificación y 5 folds para validación cruzada.

In [ ]:
# Lista de tipos de conjuntos de datos a entrenar
DATASETS_DIRS = [
    "estandarizado",
    "estandarizado_PCA80",
    "estandarizado_PCA95",
    "normalizado",
    "normalizado_PCA80",
    "normalizado_PCA95",
    "original",
    "original_PCA80",
    "original_PCA95",
]

# Número de folds en validación cruzada
K_FOLDS = 5

# Ruta al directorio de datos de entrenamiento
DATA_DIR = os.path.join("..", "data")

if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Data directory '{DATA_DIR}' does not exist. Please ensure the data is available.")

# Ruta al directorio donde se guardarán los modelos entrenados
MODELS_DIR = os.path.join("..", "models")

if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

def get_model(model_name):
    """Crea una instancia nueva del modelo especificado"""
    models = {
        "KNN": KNeighborsClassifier(),
        "SVM": SVC(probability=True),  # probability=True para obtener predict_proba
        "NaiveBayes": GaussianNB(),
        "RandomForest": RandomForestClassifier(),
    }
    return models[model_name]

# Nombres de los modelos a entrenar
MODEL_NAMES = ["KNN", "SVM", "NaiveBayes", "RandomForest"]

## Entrenamiento y Almacenamiento de Modelos

Esta sección entrena todos los modelos usando validación cruzada de 5 folds. Para cada combinación de tipo de dato y fold, se crea una instancia fresca de cada modelo, se entrena con los datos correspondientes y se serializa en formato `.pkl` usando joblib. Los modelos se organizan jerárquicamente: `models/{data_type}/{model_name}/model_fold_{n}.pkl`

### Función Auxiliar

**`train_and_save_model(X_train, y_train, model, data_type, model_name, model_iteration, model_dir)`**  
Encapsula el proceso de entrenamiento y serialización:
- Llama a `model.fit(X_train, y_train)` para entrenar con el algoritmo correspondiente
- Construye ruta jerárquica para organizar modelos por tipo de dato, algoritmo y fold
- Serializa el modelo entrenado usando `joblib.dump()` (más eficiente que pickle para objetos numpy)

In [ ]:
def train_and_save_model(X_train, y_train, model, data_type, model_name, model_iteration, model_dir):
    """Entrena un modelo y lo serializa en disco"""
    
    # Entrena el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Construye ruta jerárquica para guardar el modelo
    model_path = os.path.join(model_dir, f"{data_type}", f"{model_name}", f"model_fold_{model_iteration}.pkl")
    
    # Crea directorios intermedios si no existen
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    
    # Serializa el modelo entrenado usando joblib
    joblib.dump(model, model_path)
    

# Loop principal de entrenamiento: itera sobre tipos de datos, folds y modelos
for data_type in DATASETS_DIRS:
    
    data_path = os.path.join(DATA_DIR, data_type)
    
    if not os.path.exists(data_path):
        print(f"Data path '{data_path}' does not exist. Skipping...")
        continue
    
    for fold in range(K_FOLDS):
        
        # Carga datos de entrenamiento del fold actual
        fold_path = os.path.join(data_path, f"train_{fold + 1}_{data_type}.csv")
        
        # Carga datos una sola vez (eficiencia)
        train_data = pd.read_csv(fold_path)
        X_train = train_data.values[:, :-1]  # Features: todas las columnas excepto la última
        y_train = train_data.values[:, -1]   # Target: última columna
        
        for model_name in MODEL_NAMES:
            
            # Crea instancia fresca del modelo para evitar contaminación entre folds
            model = get_model(model_name)
            
            train_and_save_model(X_train, y_train, model, data_type, model_name, fold + 1, MODELS_DIR)